In [ ]:
import pandas as pd
from statistics import mean

In [ ]:
GNN = 0

if GNN == 1:
    model = 'MPNN' #MPNN CGCNN GCN
    ff = 'MMFF94' #UnOp GAFF MMFF94

    folder = rf'RESULTS LOCATION/{model}-{ff}/'

    train_file = folder + rf'Test_train_outputs.csv'
    val_file = folder + rf'Test_val_outputs.csv'
    test_file = folder + rf'Test_test_outputs.csv'

    train_outputs_df = pd.read_csv(train_file)
    val_outputs_df = pd.read_csv(val_file)
    test_outputs_df = pd.read_csv(test_file)
    train_val_df = pd.concat([train_outputs_df, val_outputs_df])
if GNN == 0:
    model = 'XGBoost' #XGBoost, GP, etc

    folder = rf'RESULTS LOCATION/Model Results/{model}/'
    train_file = folder + rf'Test_train_outputs-{model}.csv'
    test_file = folder + rf'Test_test_outputs-{model}.csv'

    train_outputs_df = pd.read_csv(train_file)
    test_outputs_df = pd.read_csv(test_file)

eacn_df = pd.read_csv("EACN.cvs Location/EACN.csv")

if GNN == 1:
    merged_train_df = pd.merge(train_val_df, eacn_df, left_on='ids', right_on='Molecule')
if GNN == 0:
    merged_train_df = pd.merge(train_outputs_df, eacn_df, left_on='ids', right_on='Molecule')

merged_train_df['Error'] = abs(merged_train_df['target'] - merged_train_df['prediction'])
merged_train_df = merged_train_df.drop(['Molecule', 'Target', 'SMILES'], axis=1)

merged_test_df = pd.merge(test_outputs_df, eacn_df, left_on='ids', right_on='Molecule')
merged_test_df['Error'] = abs(merged_test_df['target'] - merged_test_df['prediction'])
merged_test_df = merged_test_df.drop(['Molecule', 'Target', 'SMILES'], axis=1)

In [ ]:
merged_train_df

In [ ]:
n = 182
k = 17

In [ ]:
category_means = merged_train_df.groupby('Category')['Error'].mean()

merged_train_df = pd.merge(merged_train_df, category_means, on='Category', how='left', suffixes=('', '_mean'))
merged_train_df['Squared_Error_SSW'] = (merged_train_df['Error'] - merged_train_df['Error_mean'])**2
ssw_per_category = merged_train_df.groupby('Category')['Squared_Error_SSW'].sum()

overall_mean = merged_train_df['Error'].mean()

merged_train_df['Squared_Error_SSB'] = (merged_train_df['Error'] - overall_mean)**2
ssb_per_category = merged_train_df.groupby('Category')['Squared_Error_SSB'].sum()

SSW = sum(ssw_per_category)
SSB = sum(ssb_per_category)
print(SSB, '\n', SSW)

print('Training SSB/SSW:', ((SSB/(k-1)))/((SSW/(n-k))))

In [ ]:
category_means = merged_train_df.groupby('Category')['Error'].mean()

merged_test_df = pd.merge(merged_test_df, category_means, on='Category', how='left', suffixes=('', '_mean'))
merged_test_df['Squared_Error_SSW'] = (merged_test_df['Error'] - merged_test_df['Error_mean'])**2
ssw_per_category = merged_test_df.groupby('Category')['Squared_Error_SSW'].sum()

overall_mean = merged_test_df['Error'].mean()

merged_test_df['Squared_Error_SSB'] = (merged_test_df['Error'] - overall_mean)**2
ssb_per_category = merged_test_df.groupby('Category')['Squared_Error_SSB'].sum()

SSW = sum(ssw_per_category)
SSB = sum(ssb_per_category)
print(SSB, '\n', SSW)

print('Testing SSB/SSW:', ((SSB/(k-1)))/((SSW/(n-k))))